In [ ]:
# Do a pseudo code for vectorize photos of zeros and ones
# use SVD and PCA
# do the algebra logic by "hand", dont use scikit-learn

import numpy as np
import matplotlib.pyplot as plt
# to load images without PIL
import cv2


